In [2]:
sc

In [6]:
import csv
# data =  sc.textFile('coalesce_joined_weather_collisions_sorted_fixed.csv')
df = spark.read.load('joins_in_2015.csv', format='csv', header=True, inferSchema=False)
df.cache()


df.registerTempTable('df')



from pyspark.sql.types import IntegerType

inj_kil = [ "NUMBER_OF_CYCLIST_INJURED",
            "NUMBER_OF_CYCLIST_KILLED",
            "NUMBER_OF_MOTORIST_INJURED",
            "NUMBER_OF_MOTORIST_KILLED",
            "NUMBER_OF_PEDESTRIANS_INJURED",
            "NUMBER_OF_PEDESTRIANS_KILLED",
            "NUMBER_OF_PERSONS_INJURED",
            "NUMBER_OF_PERSONS_KILLED"
            ]

for x in inj_kil:
    df = df.withColumn(x, df[x].cast(IntegerType()))




if __name__=='__main__':
    sc = SparkContext()
    spark = SparkSession(sc)
    sqlContext = SQLContext(sc)    
    df = spark.read.load('hdfs:///user/dzeng01/project/joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)
    temp = spark.sql(query)    

In [8]:
df.take(1)

[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Traffic Control Disregarded', CONTRIBUTING_FACTOR_VEHICLE_2=None, CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'193 STREET', DATETIME=u'2015-01-01 00:20:00', LATITUDE=u'40.7518471', LOCATION=u'(40.7518471, -73.787862)', LONGITUDE=u'-73.787862', NUMBER_OF_CYCLIST_INJURED=0, NUMBER_OF_CYCLIST_KILLED=0, NUMBER_OF_MOTORIST_INJURED=0, NUMBER_OF_MOTORIST_KILLED=0, NUMBER_OF_PEDESTRIANS_INJURED=0, NUMBER_OF_PEDESTRIANS_KILLED=0, NUMBER_OF_PERSONS_INJURED=0, NUMBER_OF_PERSONS_KILLED=0, OFF_STREET_NAME=None, ON_STREET_NAME=u'47 AVENUE', UNIQUE_KEY=u'3146911', VEHICLE_TYPE_CODE_1=u'SPORT UTILITY / STATION WAGON', VEHICLE_TYPE_CODE_2=None, VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=u'11358', _c28=u'43066', Conditions=u'Clear', DateUTC=u'2015-01-01 05:51:00', DatetimeEDT=u'2015-01-01 00:51:00', Dew_PointF=u'8.1', Event

In [10]:
# df.dtypes

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
def filter_general(x):
    if x > 65:
        return float(x)
    else:
        return float(0)
    
gfg = udf(filter_general, FloatType())
newcol = df.withColumn("Dew_PointF", gfg("Dew_PointF"))
newcol.take(5)


In [32]:
# sqlContext.sql('select * from df where Rain=1')
from pyspark.sql.functions import col
from calendar import monthrange
import json


#date = ""

# Yearly statistics


def months(x):
    month = {1:"January",
             2:"Feburary",
             3:"March",
             4:"April",
             5:"May",
             6:"June",
             7:"July",
             8:"August",
             9:"September",
             10:"October",
             11:"November",
             12:"December"}
    return str(month[x])



for yr in range(15,16):
    
    #### This should be code that will read in each csv for each year and register as df.
    stats = {}
    stats["0_YEAR"] = int(str(20)+str(yr))   
    
    
    ## Overall Statistics
    stats['Overall'] = {}
    m_arr = []
    mi_arr = []
    md_arr = []
    
    """PREPROCESSING"""
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}"'.format(date)
        m_arr.append((x, sqlContext.sql(query).count())) 
    total = sum([x[1] for x in m_arr])
        
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0 or \
                                                                NUMBER_OF_PERSONS_INJURED!=0)'.format(date)
        mi_arr.append((x, sqlContext.sql(query).count()))   
    injury_total = sum([x[1] for x in mi_arr])
    
    
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0 or \
                                                                NUMBER_OF_PERSONS_KILLED!=0)'.format(date)
        md_arr.append((x, sqlContext.sql(query).count()))   
    death_total = sum([x[1] for x in md_arr])
    
    # Statistics
    stats['Overall']['Total Number of Collisions in Year'] = total
    stats['Overall']['Total Number of Collisions w/ Injury in Year'] = injury_total
    stats['Overall']['Total Number of Collisions w/ Death in Year'] = death_total
    stats['Overall']['Average Collisions Per Month'] = total // 12
    stats['Overall']['Average Collisions Per Day'] = total // 365
    stats['Overall']['Monthly Adjusted Average Weight (%)'] = [str(
                                            (months(x+1), round((float(m_arr[x][1])/float(total)*100.0),2))
                                                                ) for x in range(0,12)]
    stats['Overall']['Collisions Per Month'] = [str(
                                                (months(x+1), m_arr[x][1])
                                                ) for x in range(0,12)]
    stats['Overall']['Average Collisions Per Day (Monthly)'] = [str(
                                         (months(x+1), m_arr[x][1]/monthrange(int(str(20)+str(yr)), x+1)[1])
                                                                ) for x in range(0,12)]
    stats['Overall']['Injury Rate (Year %)'] = float(injury_total)/float(total) * 100.00
    stats['Overall']['Death Rate (Year %)'] = float(death_total)/float(total) * 100.00
    stats['Overall']['Number of Collisions w/ Injuries Per Month'] = [str(
                                                                    (months(x+1), mi_arr[x][1])
                                                                    ) for x in range(0,12)]
    stats['Overall']['Number of Collisions w/ Deaths Per Month'] = [str(
                                                                    (months(x+1), md_arr[x][1])
                                                                    ) for x in range(0,12)]
    stats['Overall']['Injury Rate Per Month (Month %)'] = [str(
                                                        (months(x+1), round((float(mi_arr[x][1])/float(m_arr[x][1]) * 100.00),2))
                                                        ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Overall']['Death Rate Per Month (Month %)'] = [str(
                                                        (months(x+1), round((float(md_arr[x][1])/float(m_arr[x][1]) * 100.00),2))
                                                        ) for x in range(0,12) if m_arr[x][1] != 0]
    
    # Rain Statistics
    stats['Rain'] = {}
    r_arr = []
    ri_arr = []
    rd_arr = []

    """PREPROCESSING"""
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}"'.format(date)
        r_arr.append((x, sqlContext.sql(query).count()))
    rain_total = sum([x[1] for x in r_arr])
    
    
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0 or \
                                                                NUMBER_OF_PERSONS_INJURED!=0)'.format(date)
        ri_arr.append((x, sqlContext.sql(query).count()))
    rain_injury_total = sum([x[1] for x in ri_arr])
    
    
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0 or \
                                                                NUMBER_OF_PERSONS_KILLED!=0)'.format(date)
        rd_arr.append((x, sqlContext.sql(query).count()))
    rain_death_total = sum([x[1] for x in rd_arr])
    
    
    
    # Statistics
    stats['Rain']['Total Number of Collisions w/ Injury & Rain in Year'] = rain_injury_total
    stats['Rain']['Total Number of Collisions w/ Death & Rain in Year'] = rain_death_total
    stats['Rain']['Injury Rate w/ Rain (Year %)'] = float(rain_injury_total)/float(rain_total) * 100.00
    stats['Rain']['Death Rate w/ Rain (Year %)'] = float(rain_death_total)/float(rain_total) * 100.00
    stats['Rain']['Total Number of Rain Collisions in Year'] = rain_total
    stats['Rain']['Percent of Collisions in Rain Over Year (%)'] = (float(rain_total) / float(total)) * 100.00
    stats['Rain']['Collisions in Rain'] = [str(
                                        (months(x+1), r_arr[x][1])
                                        ) for x in range(0,12)]
    stats['Rain']['Average Collisions in Rain Per Month'] = rain_total//12
    stats['Rain']['Number of Collisions w/ Injuries and Rain Per Month'] = [str(
                                                                            (months(x+1), ri_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Rain']['Number of Collisions w/ Deaths and Rain Per Month'] = [str(
                                                                            (months(x+1), rd_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Rain']['Percent of Collisions in Each Month That Are Rain Collisions (Rain %)'] = [str(
                                                            (months(x+1), round((float(r_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Rain']['Injury Rate Per Month w/ Rain (Month %)'] = [str(
                                                            (months(x+1), round((float(ri_arr[x][1])/float(r_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if r_arr[x][1] != 0]
    stats['Rain']['Death Rate Per Month w/ Rain (Month %)'] = [str(
                                                            (months(x+1), round((float(rd_arr[x][1])/float(r_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if r_arr[x][1] != 0]
    
    
    
    print json.dumps(stats, indent=4, sort_keys=True)
#     print stats


# print arr

# Monthly Statistics

# for x in [date]:
    
#     a = sqlContext.sql('select * from df where Rain=1')
#     a.coalesce(1).write.csv('rain', header=True)

#     a = sqlContext.sql('select * from df where Snow=1')
#     a.coalesce(1).write.csv('snow', header=True)

#     a = sqlContext.sql('select * from df where Freezing=1')
#     a.coalesce(1).write.csv('freezing', header=True)

#     a = sqlContext.sql('select * from df where LowVisibility=1')
#     a.coalesce(1).write.csv('foggy', header=True)

#     a = sqlContext.sql('select * from df where Windy=1')
#     a.coalesce(1).write.csv('windy', header=True)


# df = df.filter(df.LATITUDE != '0')

# from pyspark.sql.functions import col


# a = df.where(col("LOCATION").isNotNull())
# a.coalesce(1).write.csv('geodata', header=True)

{
    "0_YEAR": 2015, 
    "Overall": {
        "Average Collisions Per Day": 595, 
        "Average Collisions Per Day (Monthly)": [
            "('January', 520)", 
            "('Feburary', 559)", 
            "('March', 578)", 
            "('April', 559)", 
            "('May', 621)", 
            "('June', 627)", 
            "('July', 604)", 
            "('August', 612)", 
            "('September', 616)", 
            "('October', 639)", 
            "('November', 596)", 
            "('December', 608)"
        ], 
        "Average Collisions Per Month": 18118, 
        "Collisions Per Month": [
            "('January', 16122)", 
            "('Feburary', 15674)", 
            "('March', 17947)", 
            "('April', 16789)", 
            "('May', 19269)", 
            "('June', 18818)", 
            "('July', 18740)", 
            "('August', 18973)", 
            "('September', 18489)", 
            "('October', 19830)", 
            "('November', 17899)", 
            "(

In [31]:
print(float(217417) / float(35408))

6.1403355174


In [19]:
        query = 'select sum(NUMBER_OF_CYCLIST_INJURED) from df'
        a = sqlContext.sql(query)

In [14]:
df.groupBy("CONTRIBUTING_FACTOR_VEHICLE_1").count().orderBy('count', ascending=False).show()

+-----------------------------+-----+
|CONTRIBUTING_FACTOR_VEHICLE_1|count|
+-----------------------------+-----+
|                  Unspecified|97119|
|         Driver Inattentio...|31975|
|              Fatigued/Drowsy|15466|
|         Failure to Yield ...|10539|
|              Other Vehicular|10513|
|             Backing Unsafely| 7121|
|           Turning Improperly| 6542|
|           Lost Consciousness| 6280|
|         Prescription Medi...| 4625|
|         Traffic Control D...| 3136|
|         Outside Car Distr...| 2841|
|            Pavement Slippery| 2821|
|          Driver Inexperience| 2751|
|          Physical Disability| 2626|
|          Alcohol Involvement| 1826|
|            Oversized Vehicle| 1594|
|         Passenger Distrac...| 1171|
|         View Obstructed/L...|  877|
|                         null|  805|
|         Aggressive Drivin...|  795|
+-----------------------------+-----+
only showing top 20 rows



In [236]:
from pyspark.sql.functions import col
test = df.filter(col('DATETIME').like("2016-11%")).groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).rdd.map(lambda x: (x['NUMBER_OF_CYCLIST_INJURED'], x['count'])).collect()

In [237]:
test

[(0, 18847), (1, 361), (2, 25), (3, 2)]

In [233]:
df.filter(col('DATETIME').like("2016-10%"))

DataFrame[BOROUGH: string, CONTRIBUTING_FACTOR_VEHICLE_1: string, CONTRIBUTING_FACTOR_VEHICLE_2: string, CONTRIBUTING_FACTOR_VEHICLE_3: string, CONTRIBUTING_FACTOR_VEHICLE_4: string, CONTRIBUTING_FACTOR_VEHICLE_5: string, CROSS_STREET_NAME: string, DATETIME: string, LATITUDE: string, LOCATION: string, LONGITUDE: string, NUMBER_OF_CYCLIST_INJURED: int, NUMBER_OF_CYCLIST_KILLED: int, NUMBER_OF_MOTORIST_INJURED: int, NUMBER_OF_MOTORIST_KILLED: int, NUMBER_OF_PEDESTRIANS_INJURED: int, NUMBER_OF_PEDESTRIANS_KILLED: int, NUMBER_OF_PERSONS_INJURED: int, NUMBER_OF_PERSONS_KILLED: int, OFF_STREET_NAME: string, ON_STREET_NAME: string, UNIQUE_KEY: string, VEHICLE_TYPE_CODE_1: string, VEHICLE_TYPE_CODE_2: string, VEHICLE_TYPE_CODE_3: string, VEHICLE_TYPE_CODE_4: string, VEHICLE_TYPE_CODE_5: string, ZIP_CODE: string, _c28: string, Conditions: string, DateUTC: string, DatetimeEDT: string, Dew_PointF: string, Events: string, Freezing: string, Gust_SpeedMPH: string, Humidity: string, LowVisibility: st

In [242]:
df.select("LOCATION").groupBy("LOCATION").count().orderBy('count', ascending=False).show()

+--------------------+-----+
|            LOCATION|count|
+--------------------+-----+
|                null|63518|
|(40.7197624, -73....|   70|
|(40.608757, -74.0...|   60|
|(40.79522, -73.92...|   56|
|(40.6916827, -73....|   47|
|(40.6960346, -73....|   45|
|(40.8047, -73.91243)|   43|
|(40.820189, -73.8...|   37|
|(40.820305, -73.8...|   36|
|(40.7044959, -73....|   36|
|(40.79537, -73.92...|   36|
|(40.8451897, -73....|   36|
|(40.7014547, -73....|   34|
|(40.602806, -74.0...|   34|
|(40.7606, -73.96434)|   33|
|(40.798256, -73.8...|   33|
|(40.7606005, -73....|   33|
|(40.8360929, -73....|   33|
|(40.696033, -73.9...|   32|
|(40.861862, -73.9...|   32|
+--------------------+-----+
only showing top 20 rows



In [15]:
df.select("LOCATION").groupBy("LOCATION").count().orderBy('count', ascending=False).rdd.flatMap(lambda x: x).collect()

[None,
 34672,
 u'(40.8100528, -73.9251435)',
 219,
 u'(40.6528929, -73.8667311)',
 136,
 u'(40.6960346, -73.9845292)',
 135,
 u'(40.7606005, -73.9643142)',
 132,
 u'(40.7460781, -73.9749359)',
 116,
 u'(40.7595273, -73.999242)',
 112,
 u'(40.6757357, -73.8968533)',
 111,
 u'(40.7572323, -73.9897922)',
 108,
 u'(40.7203211, -73.9940403)',
 105,
 u'(40.7602226, -73.967462)',
 103,
 u'(40.6585778, -73.8906229)',
 96,
 u'(40.7584292, -73.9926328)',
 95,
 u'(40.8571285, -73.8807926)',
 92,
 u'(40.6781627, -73.8974769)',
 90,
 u'(40.8618642, -73.9127392)',
 90,
 u'(40.7832478, -73.9447234)',
 90,
 u'(40.8027515, -73.933575)',
 88,
 u'(40.759308, -73.9652931)',
 88,
 u'(40.6631828, -73.9624487)',
 87,
 u'(40.7241363, -73.9926041)',
 87,
 u'(40.7509908, -73.9906233)',
 87,
 u'(40.680015, -73.8774282)',
 85,
 u'(40.820189, -73.8906752)',
 84,
 u'(40.7548409, -73.9841185)',
 83,
 u'(40.7502939, -73.9948425)',
 82,
 u'(40.6596509, -73.7738282)',
 82,
 u'(40.7334973, -73.8703694)',
 80,
 u'(40.76

In [16]:
df.select("CONTRIBUTING_FACTOR_VEHICLE_2").distinct().rdd.flatMap(lambda x: x).collect()

[u'Following Too Closely',
 u'Traffic Control Disregarded',
 u'Accelerator Defective',
 u'Windshield Inadequate',
 u'Unsafe Speed',
 u'Shoulders Defective/Improper',
 u'Oversized Vehicle',
 u'Lane Marking Improper/Inadequate',
 u'Passing or Lane Usage Improper',
 u'Other Lighting Defects',
 None,
 u'Aggressive Driving/Road Rage',
 u'Other Vehicular',
 u'Driver Inexperience',
 u'Tow Hitch Defective',
 u'Unspecified',
 u'Pavement Defective',
 u'Prescription Medication',
 u'Cell Phone (hand-held)',
 u'View Obstructed/Limited',
 u'Lost Consciousness',
 u'Reaction to Other Uninvolved Vehicle',
 u'Fell Asleep',
 u'Tire Failure/Inadequate',
 u'Outside Car Distraction',
 u'Drugs (Illegal)',
 u'Fatigued/Drowsy',
 u'Driver Inattention/Distraction',
 u'Failure to Keep Right',
 u'Obstruction/Debris',
 u'Unsafe Lane Changing',
 u'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
 u'Passenger Distraction',
 u'Alcohol Involvement',
 u'Illness',
 u'Traffic Control Device Improper/Non-Working',
 

In [17]:
df.select("CONTRIBUTING_FACTOR_VEHICLE_1").distinct().rdd.flatMap(lambda x: x).collect()

[u'Following Too Closely',
 u'Traffic Control Disregarded',
 u'Accelerator Defective',
 u'Windshield Inadequate',
 u'Unsafe Speed',
 u'Shoulders Defective/Improper',
 u'Oversized Vehicle',
 u'Lane Marking Improper/Inadequate',
 u'Passing or Lane Usage Improper',
 u'Other Lighting Defects',
 None,
 u'Aggressive Driving/Road Rage',
 u'Other Vehicular',
 u'Driver Inexperience',
 u'Tow Hitch Defective',
 u'Unspecified',
 u'Pavement Defective',
 u'Prescription Medication',
 u'Cell Phone (hand-held)',
 u'View Obstructed/Limited',
 u'Lost Consciousness',
 u'Reaction to Other Uninvolved Vehicle',
 u'Fell Asleep',
 u'Tire Failure/Inadequate',
 u'Outside Car Distraction',
 u'Drugs (Illegal)',
 u'Fatigued/Drowsy',
 u'Driver Inattention/Distraction',
 u'Failure to Keep Right',
 u'Obstruction/Debris',
 u'Unsafe Lane Changing',
 u'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
 u'Passenger Distraction',
 u'Alcohol Involvement',
 u'Illness',
 u'Traffic Control Device Improper/Non-Working',
 